In [7]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['TORCH_USE_CUDA_DSA']   = "1"
from os import path as osp
# if 'jupyter' in os.getcwd():
#     os.chdir(osp.join(os.getcwd(), 'masterarbeit', 'code'))
import glob
import time
import argparse
from tqdm import tqdm
from tqdm.notebook import tqdm as tqdmnotebook

import multiprocessing as mp
import pickle
from pathlib import Path
from typing import Any
from typing import Callable
from typing import Optional
from typing import Tuple
from typing import Union
from typing import Dict
from typing import List
from itertools import cycle
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
mpl.rc('axes', unicode_minus=False)
preamble = r'\usepackage{amsmath}'  # LaTeX preamble command
mpl.rcParams['text.latex.preamble'] = preamble

# import seaborn as sns
import networkx as nx
import math
import numpy as np
import random
import gc

# pytorch imports
import torch
from torch import Tensor, nn, cuda
import torch.nn.functional as F
from torch.utils.data import random_split
from torch.cuda.amp import GradScaler, autocast

# pytorch geometric imports
import torch_geometric
from torch_geometric.data import Dataset
from torch_geometric.loader import DataLoader
from torch_geometric.transforms import Compose

# lightning imports
from lightning.pytorch.utilities.combined_loader import CombinedLoader

from google.cloud import storage

import sys
# Add the 'code' directory to sys.path to make the  submodules available
# sys.path.append('/home/jupyter/masterarbeit/code')

from util.utils import generate_log_name
from util.plot_utils import *

import logging as log

from data.dataset.GraphDataset import GraphDataset

from model.transform import CollapseChannels, ExtractSquare
from model.autoencoder import Autoencoder
from model.predictor import Predictor
from model.DAN import GradientReversalLayer, DomainDiscriminator

from model.criterions import WeightedMSELoss, MSLELoss, FocalLoss, ZeroInflatedLoss, CustomHuberLoss
from model.baselines import HAPredictor, ConvLSTMPredictor

from fullmodel import Model

from statsmodels.tsa.arima.model import ARIMA

In [2]:
exp_id='baselines'

In [3]:
NUM_CHANNELS: int = 2
WDW_LENGTH: list = [12, 6]
N_weeks=2
BATCH_SIZE=2
EPOCHS=2

# Constants that I may change a bit during testing
tgt: str = 'MELBOURNE'

# Constants that I don't intend to change much
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')

pre_transform = Compose([
    CollapseChannels(),
])


ds_kwargs = {
    'root_dir': 'data/raw',
    'device': device,
    'pre_transform': pre_transform,
}

# seed generator for DataLoader
torch.manual_seed(2311)

TRAIN_VAL_TEST_SPLIT_TGT = [
        N_weeks*0.04,
        0.04,
        1 - (N_weeks+1)*0.04
    ]

gs = GraphDataset(
    cities=[tgt],
    limit=None,
    **ds_kwargs,
)

train, val, test = random_split(
    gs, TRAIN_VAL_TEST_SPLIT_TGT
)

In [4]:
train_dl = DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)
val_dl   = DataLoader(  val, batch_size=BATCH_SIZE, shuffle=True)
test_dl  = DataLoader( test, batch_size=BATCH_SIZE, shuffle=False)

In [5]:
hidden_dims = [8, 16, 32, 64, 16, 2]
num_layers = len(hidden_dims)
kernel_size=3

model = ConvLSTMPredictor(
    num_channels=NUM_CHANNELS, 
    hidden_dims=hidden_dims, 
    kernel_size=kernel_size, 
    num_layers=num_layers,
).to(device)

In [ ]:
lr = 5e-6
l2_decay = 5e-7
crit = nn.MSELoss()
optimizer = torch.optim.Adam(
    [
        {'params': model.parameters()},
    ], lr=lr, weight_decay=l2_decay
)
scaler = GradScaler()
tqdm_fmt = ('{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining},'
        '{rate_fmt}{postfix}]')
tqdm_epoch = {'total': EPOCHS, 'leave': True, 'desc': 'Epochs',
              'colour': 'blue', 'bar_format': tqdm_fmt}
tqdm_batch = {'total': len(train_dl),
              'leave': True, 'desc': 'Batches', 'colour': 'green',
              'bar_format': tqdm_fmt}
 # Initialize the progress bars
pbar_epochs = tqdmnotebook(**tqdm_epoch)
epoch_handle = display(pbar_epochs, display_id='pbar_epoch')
pbar_batches = tqdmnotebook(**tqdm_batch)
batch_handle = display(pbar_batches, display_id='pbar_batch')
train_losses = []

step = 64 // BATCH_SIZE
model.train()
for epoch in range(EPOCHS):
    optimizer.zero_grad()
    for i, db in enumerate(train_dl):
        total_loss = torch.tensor(0., device=device)
        x, y = db.x, db.y
        x = gs.graph_to_image(tgt, x.view(-1, NUM_CHANNELS, WDW_LENGTH[0]))
        y = gs.graph_to_image(tgt, y.view(-1, NUM_CHANNELS, WDW_LENGTH[1]))
        
        x = torch.movedim(x, -1, 2)
        y = torch.movedim(y, -1, 2)
        
        with torch.cuda.amp.autocast():
            y_hat = model(x)
            loss = crit(y_hat, y)
        scaled_loss = scaler.scale(loss)
        train_losses.append(loss.detach().cpu().numpy())
        scaled_loss.backward()
        if (i + 1) % step == 0 or (i+1) == len(train_dl):
            scaler.step(optimizer)
            scaler.update()               
            optimizer.zero_grad()
        if i%25 == 0:
            loss_per_epoch(
                train_losses, epoch+1, specs="",
                save=False
            )
        
        pbar_batches.update(1)
    pbar_batches.reset()
    pbar_epochs.update(1)

In [ ]:
def calculate_crit(crit, y, y_hat):
    '''
    Calculate various error metrics between actual and predicted values.
    This method supports MAE, MSE, MAPE, and RMSE.

    Parameters
    ----------
    crit: {'MAE', 'MSE', 'MAPE', 'RMSE'}
        the criterion to use for calculating the error
    y: torch.Tensor
        the actual values, of shape (N, *) with N being the batch size
    y_hat: torch.Tensor
        the predicted values, of same shape as `y`

    Returns
    -------
    torch.Tensor
        the calculated error based on the specified criterion, of shape
        (N, )
    '''
    if crit == 'MAE':
        return torch.mean(torch.abs(y - y_hat), dim=1)
    elif crit == 'MSE':
        return torch.mean(torch.square(y - y_hat), dim=1)
    elif crit == 'MAPE':
        epsilon = 1e-8  # Small constant to avoid division by zero
        return torch.mean(torch.abs((y - y_hat) / (y + epsilon)), dim=1) * 100
    elif crit == 'RMSE':
        return torch.sqrt(torch.mean(torch.square(y - y_hat), dim=1))

In [ ]:
sample_limit = 1000//BATCH_SIZE
crit = ['MAE', 'RMSE']
errors = {k: torch.empty(0, device=device, dtype=torch.float16)
                 for k in crit}
if sample_limit == -1:
    sample_limit = len(test_dl)

tqdm_fmt = ('{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining},'
        '{rate_fmt}{postfix}]')
tqdm_batch = {'total': sample_limit,
              'leave': True, 'desc': 'Batches', 'colour': 'green',
              'bar_format': tqdm_fmt}
pbar_batches = tqdmnotebook(**tqdm_batch)
batch_handle = display(pbar_batches, display_id='pbar_batch')

model.eval()
with torch.no_grad():
    for i, db in enumerate(test_dl):
        x, y = db.x, db.y
        x = gs.graph_to_image(tgt, x.view(-1, NUM_CHANNELS, WDW_LENGTH[0]))
        y = gs.graph_to_image(tgt, y.view(-1, NUM_CHANNELS, WDW_LENGTH[1]))

        x = torch.movedim(x, -1, 2)
        y = torch.movedim(y, -1, 2)
        
        y /= 255

        y_hat = model(x)
        
        y = y.reshape(BATCH_SIZE, -1)
        y_hat = y_hat.reshape(BATCH_SIZE, -1)
        
        for c in crit:
            errors[c] = torch.cat(
                [errors[c], calculate_crit(c, y, y_hat)],
                dim=0
            ).detach()
        
        if i > sample_limit:
            break
        
        pbar_batches.update(1)

In [22]:
dataset = gs
N = 2
predict_window = 10
hap = HistoricalAveragePredictor(dataset=dataset, N=N, predict_window=predict_window)

In [53]:
class ARIMAPredictor:
    def __init__(self, dataset, N, predict_window, order, num_channels=2, random_start=False):
        self.dataset = dataset
        self.N = N
        self.predict_window = predict_window if predict_window is not None else len(dataset) // (240 * 7) - N
        self.order = order
        self.num_channels = num_channels
        self.random_start = random_start
        self.num_nodes, _ = self.dataset[0].x.shape
        self.snapshots = _ // self.num_channels

    def get_past_datapoints(self, index):
        past_indices = [index - 240 * 7 * week for week in range(1, self.N + 1)]
        past_datapoints = [self.dataset[i].x.view(-1, self.num_channels, self.snapshots) for i in past_indices]
        stacked_past = torch.stack(past_datapoints).numpy()
        return stacked_past

    def fit_predict_arima(self, node_channel_snapshot_data):
        node, channel, snapshot, past_datapoints = node_channel_snapshot_data
        model = ARIMA(past_datapoints, order=self.order)
        model_fit = model.fit()
        fc = model_fit.forecast(steps=1)[0]
        return node, channel, snapshot, fc

    def predict(self):
        crit = ['MAE', 'RMSE']
        errors = {k: torch.empty(0, device='cpu', dtype=torch.float16) for k in crit}
        ds_start = 240 * 7 * self.N
        ds_end = ds_start + 240 * 7 * self.predict_window

        if self.random_start:
            ds_start = random.randint(ds_start, len(self.dataset) - 240 * 7 * self.predict_window)
            print(f'Random start: {ds_start}')

        tqdm_args = {'total': ds_end - ds_start, 'leave': True, 'desc': 'Predicting', 'colour': 'blue'}
        pbar = tqdm(**tqdm_args)

        # Parallel processing setup
        pool = mp.Pool(1)
        for i in range(ds_start, ds_end):
            actual = self.dataset[i].x.view(-1, self.num_channels, self.snapshots).numpy()
            prediction = np.zeros_like(actual)

            # Prepare data for parallel processing
            parallel_data = []
            for node in range(self.num_nodes):
                print(node)
                for channel in range(self.num_channels):
                    for snapshot in range(self.snapshots):
                        past_datapoints = self.get_past_datapoints(i)[:, node, channel, snapshot]
                        parallel_data.append((node, channel, snapshot, past_datapoints))
            # Parallel ARIMA fitting and forecasting
            forecasts = pool.map(self.fit_predict_arima, parallel_data)
            for node, channel, snapshot, fc in forecasts:
                prediction[node, channel, snapshot] = fc

            for c in crit:
                errors[c] = torch.cat([errors[c], self.calculate_crit(c, actual, prediction)], dim=0).detach()

            pbar.update(1)

        pool.close()
        pool.join()
        
        return errors
                                       
    @staticmethod
    def calculate_crit(crit, y, y_hat):
        '''
        Calculate various error metrics between actual and predicted values.
        This method supports MAE, MSE, MAPE, and RMSE.
    
        Parameters
        ----------
        crit: {'MAE', 'MSE', 'MAPE', 'RMSE'}
            the criterion to use for calculating the error
        y: np.ndarray
            the actual values, of shape (N, *) with N being the batch size
        y_hat: np.ndarray
            the predicted values, of same shape as `y`
    
        Returns
        -------
        torch.Tensor
            the calculated error based on the specified criterion, of shape
            (N, )
        '''
        y = torch.tensor(y)
        y_hat = torch.tensor(y_hat)
        if crit == 'MAE':
            return torch.mean(torch.abs(y - y_hat), dim=1)
        elif crit == 'MSE':
            return torch.mean(torch.square(y - y_hat), dim=1)
        elif crit == 'MAPE':
            epsilon = 1e-8  # Small constant to avoid division by zero
            return torch.mean(torch.abs((y - y_hat) / (y + epsilon)), dim=1) * 100
        elif crit == 'RMSE':
            return torch.sqrt(torch.mean(torch.square(y - y_hat), dim=1))

In [54]:
dataset = gs
N = 2
predict_window=1

arima = ARIMAPredictor(
    dataset=dataset,
    N=N,
    predict_window=predict_window,
    order=(0, 2, 2),
    random_start=True
)

errors = arima.predict()

Random start: 15562







Predicting: 0it [00:00, ?it/s]


In [74]:
from model.baselines import HAPredictor
from typing import Union, List
import torch
from torch import nn
from statsmodels.tsa.arima.model import ARIMA
from tqdm import tqdm
from tqdm.notebook import tqdm as tqdmnotebook
import numpy as np

class HAPredictor:
    def __init__(
        self,
        dataset,
        N,
        predict_window,
    ):
        '''
        Parameters
        ----------
        dataset : GraphDataset
        N : int
            number of weeks to consider
        predict_window : int
            number of weeks to predict
        '''
        self.dataset = dataset
        self.N = N  # number of weeks to consider
        if predict_window is None or predict_window == -1:
            self.predict_window = len(dataset) // (240*7) - N
        else:
            self.predict_window = predict_window

    def get_past_datapoints(self, index):
        past_indices = [index - 240 * 7 * week for week in range(1, self.N + 1)]
        past_datapoints = [self.dataset[i].x for i in past_indices]
        stacked_past = torch.stack(past_datapoints).numpy()
        
        return stacked_past

    def predict(self):
        predictions = []
        crit = ['MAE', 'RMSE']
        errors = {k: torch.empty(0, device='cpu', dtype=torch.float16)
                 for k in crit}
        
        ds_start = 240*7*self.N
        ds_end   = int(240*7*(self.N+self.predict_window))
        
        tqdm_fmt = ('{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining},'
                '{rate_fmt}{postfix}]')
        tqdm_args = {'total': ds_end, 'leave': True, 'desc': 'Epochs',
                      'colour': 'blue', 'bar_format': tqdm_fmt, 'initial': ds_start}
        
        # Initialize the progress bars
        pbar = tqdmnotebook(**tqdm_args)
        epoch_handle = display(pbar, display_id='pbar')

        # Start from the Nth week to ensure we have past data
        # for i in range(240 * 7 * self.N, len(self.dataset)):
        for i in range(ds_start, ds_end):
            past_datapoints = self.get_past_datapoints(i)
            prediction = np.mean(past_datapoints, axis=0)
            actual = self.dataset[i].x.numpy()
            actual_nonzero = actual[actual.nonzero()]
            pred_nonzero   = prediction[actual.nonzero()]

            for c in crit:
                metric = self.calculate_crit(c, actual_nonzero, pred_nonzero)
                errors[c] = torch.cat(
                    [
                        errors[c],
                        metric.reshape(1)
                    ],
                dim=0).detach()
            pbar.update(1)

        return errors
                                       
    @staticmethod
    def calculate_crit(crit, y, y_hat):
        '''
        Calculate various error metrics between actual and predicted values.
        This method supports MAE, MSE, MAPE, and RMSE.
    
        Parameters
        ----------
        crit: {'MAE', 'MSE', 'MAPE', 'RMSE'}
            the criterion to use for calculating the error
        y: np.ndarray
            the actual values, of shape (N, *) with N being the batch size
        y_hat: np.ndarray
            the predicted values, of same shape as `y`
    
        Returns
        -------
        torch.Tensor
            the calculated error based on the specified criterion, of shape
            (N, )
        '''
        y = torch.tensor(y)
        y_hat = torch.tensor(y_hat)
        assert y.shape == y_hat.shape, "Misshape"
        if crit == 'MAE':
            return torch.mean(torch.abs(y - y_hat), dim=0)
        elif crit == 'MSE':
            return torch.mean(torch.square(y - y_hat), dim=1)
        elif crit == 'MAPE':
            epsilon = 1e-8  # Small constant to avoid division by zero
            return torch.mean(torch.abs((y - y_hat) / (y + epsilon)), dim=1) * 100
        elif crit == 'RMSE':
            return torch.sqrt(torch.mean(torch.square(y - y_hat), dim=0))

# Usage
dataset = gs
N = 2
hap = HAPredictor(dataset, N, predict_window=1)
errors = hap.predict()

Epochs:  67%|######6   | 3360/5040 [00:00<?,?it/s]

In [75]:
errors['MAE'].median()

tensor(0.0888)

In [76]:
errors['RMSE'].median()

tensor(0.1150)

In [41]:
errors['MAE'][12323323]

tensor(0.)

4177236

In [46]:
int(torch.count_nonzero(errors['MAE']))/int(errors['MAE'].shape[0])

0.21177497657780428